In [1]:
import numpy as np
import scanpy.api as sc
import pandas as pd

import os
import pandas as pd
import mnnpy
import matplotlib

In [2]:
import scanorama

In [3]:
human_dge = sc.AnnData(pd.read_csv("./Human/Human_cell100_orth.csv").fillna(0).T )
mouse_dge = sc.AnnData(pd.read_csv("./Mouse/Mouse_cell100_orth.csv").fillna(0).T )

In [4]:
print(mouse_dge.shape, human_dge.shape)

(21702, 16679) (25403, 16679)


In [ ]:
intergrated, corrected = scanorama.correct_scanpy([human_dge, mouse_dge], return_dimred=True)
corrected_human_dge, corrected_mouse_dge = corrected
corrected_merged_dge = corrected_human_dge.concatenate(corrected_mouse_dge, join='outer')

Found 16679 genes among all datasets
[[0.         0.35775505]
 [0.         0.        ]]
Processing datasets (0, 1)


/home/ggj/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:512: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
/home/ggj/anaconda3/lib/python3.7/site-packages/scipy/sparse/lil.py:514: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):


In [6]:
corrected_merged_dge

AnnData object with n_obs × n_vars = 47105 × 2760 
    obs: 'batch'

In [67]:
corrected_merged_dge.write('./corrected_merged_mouse_human_dge.h5ad')

In [7]:
##scanpy_visualization
sc.pp.normalize_per_cell(corrected_merged_dge, counts_per_cell_after=1e6)
sc.pp.scale(corrected_merged_dge)

In [8]:
sc.tl.pca(corrected_merged_dge)

In [17]:
sc.pl.pca_variance_ratio(corrected_merged_dge, n_pcs=50, log=False, save="_pca.png")

saving figure to file ./figures/pca_variance_ratio_pca.png


/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [18]:
sc.pp.neighbors(corrected_merged_dge, n_neighbors=10, n_pcs=40)

In [19]:
sc.tl.tsne(corrected_merged_dge, n_pcs=40, learning_rate=1000, perplexity=100, early_exaggeration=20, use_fast_tsne=True, n_jobs=25)

In [22]:
col1=["#2685cb", "#fd8d14"]
sc.pl.tsne(corrected_merged_dge, color='batch', palette=col1, save='_batch_merged.pdf')

saving figure to file ./figures/tsne_batch_merged.pdf


/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [29]:
corrected_merged_dge.obs_names

Index(['0-0', '1-0', '2-0', '3-0', '4-0', '5-0', '6-0', '7-0', '8-0', '9-0',
       ...
       '21692-1', '21693-1', '21694-1', '21695-1', '21696-1', '21697-1',
       '21698-1', '21699-1', '21700-1', '21701-1'],
      dtype='object', length=47105)

In [23]:
mouse_anno = pd.read_table("./Mouse/Mouse.SRS.phe2", header=0)
human_anno = pd.read_table("./Human/Human.SRS.phe2", header=0)

In [24]:
print(mouse_anno.shape, human_anno.shape)

(21702, 3) (25403, 3)


In [25]:
merged_anno = human_anno.append(mouse_anno, sort=False)
merged_anno.head()

,CellID,Tissue,Celltype
0,AdultDuodenum_1.AGGACTACAATACCGCTA,Human,Plasmocyte A.h
1,AdultSpleenParenchyma_1.AGATGGGTTGCCACAATA,Human,Plasmocyte A.h
2,AdultColon_1.CAAAGTACCTGAAACCTA,Human,Plasmocyte A.h
3,AdultEsophagus_1.CGCACCCTGAAATGCGGA,Human,Plasmocyte A.h
4,AdultSpleen_1.TGCAATACTTATGTCCCG,Human,Plasmocyte A.h


In [36]:
corrected_merged_dge.obs.to_csv("./merged/corrected_merged_dgeobs.csv")

In [88]:
celltype_anno = pd.read_table("./merged/438celltype-NEW-20190728.annotation", sep=',')
celltype_anno['Cluster'] = celltype_anno['Cluster'].apply(lambda x: x.split('.')[0])
celltype_anno['Sub_Cluster'] = celltype_anno['Sub_Cluster'].apply(lambda x: x.split('.')[0])
celltype_anno.head()

,Celltype,Species,Cluster,Sub_Cluster
0,Apelin cells(H).z,Zebrafish,Apelin,Apelin
1,Apelin cells(P).z,Zebrafish,Apelin,Apelin
2,Brown.fat.m,Mouse,BrownFat,Muscle
3,Cnidocytean.n,Nematostella,Cnidocytean,Cnidocytean
4,Coelomocytes.c,Celegans,Coelomocytes,Coelomocytes


In [89]:
merged_celltype_anno = pd.merge(merged_anno, celltype_anno, on=['Celltype'], how='inner')
merged_celltype_anno.head()

,CellID,Tissue,Celltype,Species,Cluster,Sub_Cluster
0,AdultDuodenum_1.AGGACTACAATACCGCTA,Human,Plasmocyte A.h,Human,Immune,Plasmocyte
1,AdultSpleenParenchyma_1.AGATGGGTTGCCACAATA,Human,Plasmocyte A.h,Human,Immune,Plasmocyte
2,AdultColon_1.CAAAGTACCTGAAACCTA,Human,Plasmocyte A.h,Human,Immune,Plasmocyte
3,AdultEsophagus_1.CGCACCCTGAAATGCGGA,Human,Plasmocyte A.h,Human,Immune,Plasmocyte
4,AdultSpleen_1.TGCAATACTTATGTCCCG,Human,Plasmocyte A.h,Human,Immune,Plasmocyte


In [90]:
corrected_merged_dge.obs['cellID']=merged_celltype_anno['CellID'].values
corrected_merged_dge.obs['Celltype']=merged_celltype_anno['Celltype'].values
corrected_merged_dge.obs['Cluster']=merged_celltype_anno['Cluster'].values
corrected_merged_dge.obs['Sub_Cluster']=merged_celltype_anno['Sub_Cluster'].values
corrected_merged_dge.obs['Species']=merged_celltype_anno['Species'].values

In [85]:
corrected_merged_dge.obs.to_csv("./merged/corrected_merged_dgeobs_annotated.csv")

In [53]:
#col2 = ["#E41A1C","#E7298A","#4DAF4A","#6A3D9A","#984EA3","#1B9E77","#66A61E","#D95F02","#A6761D","#E6AB02","#7570B3", '#D9D9D9']
#col2 = ['#D9D9D9','#D9D9D9','#D9D9D9','#D9D9D9','#D9D9D9','#D9D9D9','#D9D9D9','#D9D9D9','#D9D9D9','#D9D9D9', '#D9D9D9',"#E41A1C"]
sc.pl.tsne(corrected_merged_dge, color='Cluster', palette=col2, save='_merged_celltype.png')

... storing 'cellID' as categorical
... storing 'Celltype' as categorical
... storing 'Cluster' as categorical
... storing 'Species' as categorical


saving figure to file ./figures/tsne_merged_celltype.png


/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [114]:
sc.pl.tsne(corrected_merged_dge, color='Sub_Cluster', save='_merged_SubCelltype.pdf')

saving figure to file ./figures/tsne_merged_SubCelltype.pdf


/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/

In [110]:
#sc.pl.tsne(corrected_merged_dge, color='Sub_Cluster', save='_merged_SubCelltype.pdf')
sc.pl.tsne(corrected_merged_dge, color='Cluster', groups=['Secretory', 'Endothelial'], palette=['#E41A1C',"#66A61E"], save= '_Secretory_Endothelial' + '_SubCelltype.pdf')
#for subcluster in corrected_merged_dge.obs['Cluster'].drop_duplicates():
#   sc.pl.tsne(corrected_merged_dge, color='Cluster', groups=[subcluster], palette=['#E41A1C'], save= '_' + subcluster + '_Celltype.pdf')

saving figure to file ./figures/tsne_Secretory_Endothelial_SubCelltype.pdf


/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/

In [55]:
col1=["#2685cb", "#fd8d14"]
sc.pl.tsne(corrected_merged_dge, color='Species', palette=col1, save='_batch_merged.pdf')

saving figure to file ./figures/tsne_batch_merged.pdf


/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [61]:
sc.tl.umap(corrected_merged_dge)

In [62]:
sc.pl.umap(corrected_merged_dge, color='Cluster', palette=col2, save='_umap_merged_celltype.pdf')

saving figure to file ./figures/umap_umap_merged_celltype.pdf


/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [68]:
sc.tl.louvain(corrected_merged_dge)
sc.tl.rank_genes_groups(corrected_merged_dge, 'louvain', method='wilcoxon')

In [70]:
result = corrected_merged_dge.uns['rank_genes_groups']
groups = result['names'].dtype.names
marker_genes = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']})

In [72]:
marker_genes.to_csv("./merged/marker_genes.csv")

In [74]:
print(corrected_merged_dge.obs_keys)
sc.pl.tsne(corrected_merged_dge, color='louvain', save='_louvain_merged.pdf')

<bound method AnnData.obs_keys of AnnData object with n_obs × n_vars = 47105 × 2760 
    obs: 'batch', 'n_counts', 'cellID', 'Celltype', 'Cluster', 'Species', 'louvain'
    uns: 'pca', 'neighbors', 'batch_colors', 'Cluster_colors', 'Species_colors', 'louvain', 'rank_genes_groups'
    obsm: 'X_pca', 'X_tsne'
    varm: 'PCs'>
saving figure to file ./figures/tsne_louvain_merged.pdf


/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())
/home/ggj/anaconda3/lib/python3.7/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


In [115]:
corrected_merged_dge.write('./merged/corrected_merged_mouse_human_dge.h5ad')